### Loading the data

In [1]:
from dataset import Dataset

dataset = Dataset()
dataset.call()
dataset.inspect()

X_train shape: (15984, 64, 64, 3)
15984 train samples
1776 test samples
8792 images of vehicles
8968 images of non vehicles
